<a href="https://colab.research.google.com/github/Kingadamy/210-demo-stl-list/blob/main/Meeting_2_Volatility_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setting Up Dependencies**

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

**Loading and Pre-Analysis**

In [ ]:
ticker = "SPY"
df = yf.download(ticker, start="2022-01-01", end="2025-01-01")

/tmp/ipython-input-4165379074.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2022-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed


In [ ]:
df

Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
2022-01-03,453.210388,453.343222,449.548364,451.872697,72668200
2022-01-04,453.058624,455.364023,451.189655,454.642990,71178700
2022-01-05,444.358948,453.466614,444.264071,452.688661,104538900
2022-01-06,443.941467,446.673770,441.560185,443.894043,86858900
2022-01-07,442.186310,445.136827,440.820159,443.950934,85111600
...,...,...,...,...,...
2024-12-24,596.076965,596.116657,590.297589,590.882491,33160100
2024-12-26,596.116638,597.246690,592.884946,594.292594,41219100


**Feature Engineering (5 inputs)**

In [ ]:
df["PCT_return"] = df["Close"].pct_change()
df

Price,Close,High,Low,Open,Volume,PCT_return
Ticker,SPY,SPY,SPY,SPY,SPY,
Date,,,,,,
2022-01-03,453.210388,453.343222,449.548364,451.872697,72668200,NaN
2022-01-04,453.058624,455.364023,451.189655,454.642990,71178700,-0.000335
2022-01-05,444.358948,453.466614,444.264071,452.688661,104538900,-0.019202
2022-01-06,443.941467,446.673770,441.560185,443.894043,86858900,-0.000940
2022-01-07,442.186310,445.136827,440.820159,443.950934,85111600,-0.003954
...,...,...,...,...,...,...
2024-12-24,596.076965,596.116657,590.297589,590.882491,33160100,0.011115
2024-12-26,596.116638,597.246690,592.884946,594.292594,41219100,0.000067


In [ ]:
df["3day_volatility"] = df["PCT_return"].rolling(window=3).std()
df

Price,Close,High,Low,Open,Volume,PCT_return,3day_volatility
Ticker,SPY,SPY,SPY,SPY,SPY,,
Date,,,,,,,
2022-01-03,453.210388,453.343222,449.548364,451.872697,72668200,NaN,NaN
2022-01-04,453.058624,455.364023,451.189655,454.642990,71178700,-0.000335,NaN
2022-01-05,444.358948,453.466614,444.264071,452.688661,104538900,-0.019202,NaN
2022-01-06,443.941467,446.673770,441.560185,443.894043,86858900,-0.000940,0.010723
2022-01-07,442.186310,445.136827,440.820159,443.950934,85111600,-0.003954,0.009791
...,...,...,...,...,...,...,...
2024-12-24,596.076965,596.116657,590.297589,590.882491,33160100,0.011115,0.003250
2024-12-26,596.116638,597.246690,592.884946,594.292594,41219100,0.000067,0.005529


In [ ]:
df["volume_norm"] = df["Volume"].rolling(3).mean() / df['Volume'].rolling(20).mean()
df

Price,Close,High,Low,Open,Volume,PCT_return,3day_volatility,volume_norm
Ticker,SPY,SPY,SPY,SPY,SPY,,,
Date,,,,,,,,
2022-01-03,453.210388,453.343222,449.548364,451.872697,72668200,NaN,NaN,NaN
2022-01-04,453.058624,455.364023,451.189655,454.642990,71178700,-0.000335,NaN,NaN
2022-01-05,444.358948,453.466614,444.264071,452.688661,104538900,-0.019202,NaN,NaN
2022-01-06,443.941467,446.673770,441.560185,443.894043,86858900,-0.000940,0.010723,NaN
2022-01-07,442.186310,445.136827,440.820159,443.950934,85111600,-0.003954,0.009791,NaN
...,...,...,...,...,...,...,...,...
2024-12-24,596.076965,596.116657,590.297589,590.882491,33160100,0.011115,0.003250,1.520194
2024-12-26,596.116638,597.246690,592.884946,594.292594,41219100,0.000067,0.005529,0.931231


In [ ]:
df["fast_sma"] = df["Close"].rolling(5).mean()
df["slow_sma"] = df["Close"].rolling(20).mean()

In [ ]:
df["Slow_Slope"] = (df["slow_sma"] - df["slow_sma"].shift(3) / (3 * df["slow_sma"].shift(3)))
df["Fast_Slope"] = (df["fast_sma"] - df["fast_sma"].shift(3) / (3 * df["fast_sma"].shift(3)))
df

Price,Close,High,Low,Open,Volume,PCT_return,3day_volatility,volume_norm,fast_sma,slow_sma,Slow_Slope,Fast_Slope
Ticker,SPY,SPY,SPY,SPY,SPY,,,,,,,
Date,,,,,,,,,,,,
2022-01-03,453.210388,453.343222,449.548364,451.872697,72668200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,453.058624,455.364023,451.189655,454.642990,71178700,-0.000335,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,444.358948,453.466614,444.264071,452.688661,104538900,-0.019202,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,443.941467,446.673770,441.560185,443.894043,86858900,-0.000940,0.010723,NaN,NaN,NaN,NaN,NaN
2022-01-07,442.186310,445.136827,440.820159,443.950934,85111600,-0.003954,0.009791,NaN,447.351147,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,596.076965,596.116657,590.297589,590.882491,33160100,0.011115,0.003250,1.520194,585.982886,594.363748,594.030415,585.649552
2024-12-26,596.116638,597.246690,592.884946,594.292594,41219100,0.000067,0.005529,0.931231,589.358630,594.497815,594.164482,589.025297


In [ ]:
df['DOW-T'] = df.index.dayofweek
df

Price,Close,High,Low,Open,Volume,PCT_return,3day_volatility,volume_norm,fast_sma,slow_sma,Slow_Slope,Fast_Slope,DOW-T
Ticker,SPY,SPY,SPY,SPY,SPY,,,,,,,,
Date,,,,,,,,,,,,,
2022-01-03,453.210388,453.343222,449.548364,451.872697,72668200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-01-04,453.058624,455.364023,451.189655,454.642990,71178700,-0.000335,NaN,NaN,NaN,NaN,NaN,NaN,1
2022-01-05,444.358948,453.466614,444.264071,452.688661,104538900,-0.019202,NaN,NaN,NaN,NaN,NaN,NaN,2
2022-01-06,443.941467,446.673770,441.560185,443.894043,86858900,-0.000940,0.010723,NaN,NaN,NaN,NaN,NaN,3
2022-01-07,442.186310,445.136827,440.820159,443.950934,85111600,-0.003954,0.009791,NaN,447.351147,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,596.076965,596.116657,590.297589,590.882491,33160100,0.011115,0.003250,1.520194,585.982886,594.363748,594.030415,585.649552,1
2024-12-26,596.116638,597.246690,592.884946,594.292594,41219100,0.000067,0.005529,0.931231,589.358630,594.497815,594.164482,589.025297,3


In [ ]:
real_vol = df["PCT_return"].rolling(10).std()
future_vol = real_vol.shift(-1)
q = future_vol.quantile([1/3, 2/3]).values
df["target_cls"] = np.select([future_vol <= q[0], (future_vol > q[0]) & (future_vol <= q[1]), future_vol > q[1]], [0, 1, 2], default=np.nan).astype('float')

In [ ]:
df

Price,Close,High,Low,Open,Volume,PCT_return,3day_volatility,volume_norm,fast_sma,slow_sma,Slow_Slope,Fast_Slope,DOW-T,target_cls
Ticker,SPY,SPY,SPY,SPY,SPY,,,,,,,,,
Date,,,,,,,,,,,,,,
2022-01-03,453.210388,453.343222,449.548364,451.872697,72668200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2022-01-04,453.058624,455.364023,451.189655,454.642990,71178700,-0.000335,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
2022-01-05,444.358948,453.466614,444.264071,452.688661,104538900,-0.019202,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
2022-01-06,443.941467,446.673770,441.560185,443.894043,86858900,-0.000940,0.010723,NaN,NaN,NaN,NaN,NaN,3,NaN
2022-01-07,442.186310,445.136827,440.820159,443.950934,85111600,-0.003954,0.009791,NaN,447.351147,NaN,NaN,NaN,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,596.076965,596.116657,590.297589,590.882491,33160100,0.011115,0.003250,1.520194,585.982886,594.363748,594.030415,585.649552,1,2.0
2024-12-26,596.116638,597.246690,592.884946,594.292594,41219100,0.000067,0.005529,0.931231,589.358630,594.497815,594.164482,589.025297,3,2.0


In [ ]:
features = ['3day_volatility', 'volume_norm', 'Slow_Slope', 'Fast_Slope', 'DOW-T']
training_df = df[features + ['target_cls']].dropna()
training_df

Price,3day_volatility,volume_norm,Slow_Slope,Fast_Slope,DOW-T,target_cls
Ticker,,,,,,
Date,,,,,,
2022-02-03,0.018385,0.920992,428.970413,426.352922,3,2.0
2022-02-04,0.017908,0.897985,428.057749,427.633683,4,2.0
2022-02-07,0.014550,0.814718,427.164540,427.130870,0,2.0
2022-02-08,0.005857,0.731445,426.473397,426.749491,1,2.0
2022-02-09,0.009040,0.660152,425.894209,426.785545,2,2.0
...,...,...,...,...,...,...
2024-12-23,0.006160,1.872330,593.744209,585.840470,0,2.0
2024-12-24,0.003250,1.520194,594.030415,585.649552,1,2.0


**Model #1 Training (Logistic Regression)**

**Model #1 Analysis (Logistic Regression)**

**Model #2 Training  (XGBoost)**

**Model #2 Analysis (XGBoost)**